In [ ]:
from adler.objectdata.AdlerPlanetoid import AdlerPlanetoid
from adler.science.PhaseCurve import PhaseCurve
from adler.utilities.plotting_utilities import plot_errorbar
import adler.utilities.science_utilities as sci_utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u

In [ ]:
# notebook to show adler searching for outlying photometry
# we could simply use the adler api to do this, but let's demo the CLI that would be running on a server

In [ ]:
# ssObjectId of object to analyse
ssoid1 = "6098332225018"  # good test object
ssoid2 = "6098332225018000"  # fake outburst object
filt = "r"

In [ ]:
# here we use an offline SQL database which contains the observations of the sso
fname = "/home/jrob/lsst-adler/notebooks/gen_test_data/adler_demo_testing_database.db"
planetoid1 = AdlerPlanetoid.construct_from_SQL(ssoid1, sql_filename=fname)
planetoid2 = AdlerPlanetoid.construct_from_SQL(ssoid2, sql_filename=fname)

# or query DP0.3 directly when on RSP
# planetoid1 = AdlerPlanetoid.construct_from_RSP(ssoid1)

In [ ]:
planetoid1

In [ ]:
planetoid1.__dict__

In [ ]:
fig = plot_errorbar(planetoid1, filt_list=["r"], x_plot="midPointMjdTai", y_plot="reduced_mag")
fig = plot_errorbar(planetoid1, filt_list=["r"], x_plot="phaseAngle", y_plot="reduced_mag")

In [ ]:
fig = plot_errorbar(planetoid2, filt_list=["r"], x_plot="midPointMjdTai", y_plot="reduced_mag")
fig = plot_errorbar(planetoid2, filt_list=["r"], x_plot="phaseAngle", y_plot="reduced_mag")

In [ ]:
# inspect observations

In [ ]:
obs = planetoid1.observations_in_filter(filt)

In [ ]:
df_obs = pd.DataFrame(obs.__dict__)

In [ ]:
df_obs

In [ ]:
tmin = np.amin(np.floor(df_obs["midPointMjdTai"]))  # mjd
tmax = np.amax(np.floor(df_obs["midPointMjdTai"])) + 1  # mjd
tmin, tmax

In [ ]:
# cumulative data in filter
x_plot = "midPointMjdTai"
df_plot = df_obs.sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

bins = np.arange(tmin, tmax + 1)

values, base = np.histogram(df_plot[x_plot], bins=bins)
cumulative = np.cumsum(values)
ax1.plot(base[:-1] - base[0], cumulative, label=filt)

data_mask = np.diff(cumulative) > 0
data_nights = base[1:-1][data_mask]
N_data = cumulative[1:][data_mask]

ax1.scatter(data_nights - data_nights[0], N_data)

ax1.set_xlabel(x_plot)
ax1.set_ylabel("number")
ax1.legend()

plt.show()

In [ ]:
# number of data points per night of new data
np.diff(N_data)

In [ ]:
# nights when new data arrives
data_nights

In [ ]:
# write commands to simulate adler cli searching for phase curve outliers on incoming data

In [ ]:
# cmd_list = []
# outpath = "~/lsst-adler/logging"
# cmd = "adler -s {} -f {} -o {}".format(ssoid2,filt,outpath)

# for t0 in data_nights:
#     t1 = t0+1

#     mask = (df_obs["midPointMjdTai"]<t1)
#     _df_obs = df_obs[mask]
#     # print(t0,t1, len(_df_obs))
#     _cmd = cmd + " -i {} -d {} {}".format(fname, tmin,t1)
#     print(_cmd)
#     cmd_list.append(_cmd)

In [ ]:
# cmd_file = "../adler_demo_cli/adler_{}.sh".format(ssoid2)
# with open(cmd_file,"w") as f:
#     f.write("#!/usr/bin/env bash\n")
#     f.write("\n".join(cmd_list))

In [ ]:
# using the adler api, search for outliers

In [ ]:
N_pc_fit = 10
diff_cut = 1.0
filt = "r"

# get the filter SSObject metadata
sso = planetoid2.SSObject_in_filter(filt)

# get the observations
obs = planetoid2.observations_in_filter(filt)
df_obs = pd.DataFrame(obs.__dict__)

# get the LSST phase curve filter model
pc = PhaseCurve(
    abs_mag=sso.H * u.mag,
    phase_param=0.62,
    model_name="HG12_Pen16",
)

j = 0
for i, t0 in enumerate(data_nights):
    t1 = t0 + 1

    # load the df_obs outlier column here
    outlier_file_load = "data/outlier_{}_{}.npy".format(ssoid2, int(data_nights[i - 1]))
    outlier_file_save = "data/outlier_{}_{}".format(ssoid2, int(t0))
    if j == 0:
        outliers = [False] * len(df_obs)
    else:
        outliers = np.load(outlier_file_load)
    df_obs["outlier"] = outliers

    t_mask = df_obs["midPointMjdTai"] < t1
    _df_obs = df_obs[t_mask]
    mask = _df_obs["midPointMjdTai"] < t0
    df_obs_old = _df_obs[(mask) & (_df_obs["outlier"] == False)]
    df_obs_new = _df_obs[~mask]

    if len(df_obs_old) < 2:
        print("continue")
        continue
    else:
        j = 1

    if len(df_obs_old) < N_pc_fit:
        # use an assumed value of G12 until more data is available
        pc.model_function.G12.fixed = True
    else:
        pc.model_function.G12.fixed = False

    # do a simple HG12_Pen16 fit to the past data
    pc_fit = pc.FitModel(
        np.array(df_obs_old["phaseAngle"]) * u.deg,
        np.array(df_obs_old["reduced_mag"]) * u.mag,
        np.array(df_obs_old["magErr"]) * u.mag,
    )
    pc_fit = pc.InitModelSbpy(pc_fit)

    print(t0, t1, len(df_obs_old), len(df_obs_new), pc_fit.model_function.H, pc_fit.model_function.G12)

    # find outliers in new data
    # calculate data - model residuals
    res = (np.array(df_obs_new["reduced_mag"]) * u.mag) - pc_fit.ReducedMag(
        np.array(df_obs_new["phaseAngle"]) * u.deg
    )

    outlier_flag = sci_utils.outlier_diff(res.value, diff_cut=diff_cut)
    print(outlier_flag)

    df_obs.loc[(t_mask & ~mask), "outlier"] = outlier_flag

    # save the df_obs outlier column, in reality it would have been saved as part of AdlerData on the server
    print("save {}".format(outlier_file_save))
    np.save(outlier_file_save, np.array(df_obs["outlier"]))

    # make a plot
    fig = plot_errorbar(planetoid2, filt_list=[])
    ax1 = fig.axes[0]
    ax1.scatter(df_obs_old["phaseAngle"], df_obs_old["reduced_mag"], c="C0")
    alpha = np.linspace(0, np.amax(obs.phaseAngle)) * u.deg
    ax1.plot(alpha.value, pc_fit.ReducedMag(alpha).value, label="t={}".format(int(t0)))
    ax1.scatter(
        df_obs_new["phaseAngle"], df_obs_new["reduced_mag"], edgecolor="r", facecolor="none", zorder=3
    )

    # ax1.scatter(df_obs_new.loc[outlier_flag]["phaseAngle"], df_obs_new.loc[outlier_flag]["reduced_mag"], c = "r", marker = "x", s= 75, zorder = 3)
    out_mask = df_obs["outlier"] == True
    ax1.scatter(
        df_obs.loc[out_mask]["phaseAngle"],
        df_obs.loc[out_mask]["reduced_mag"],
        c="r",
        marker="x",
        s=75,
        zorder=3,
    )
    fig_file = "plots/phase_curve_{}_{}.png".format(ssoid2, int(t0))
    print(fig_file)
    fig = plot_errorbar(planetoid2, fig=fig, filename=fig_file)
    plt.show()

In [ ]:
df_obs[df_obs["outlier"] == True]